In [3]:
from pyspark import SparkContext
from math import sin,cos,sqrt,atan2,radians
sc = SparkContext.getOrCreate()
rdd = sc.textFile("/home/ercan/Desktop/hadoop/datasets/Capitals.txt")

def splitAndMakeTuple(satir):
    arr = satir.split("\t")
    country = arr[2].strip()
    lat = float(arr[3].strip())
    long = float(arr[4].strip())
    return (country,(lat,long))

rdd  = rdd.map(lambda line:line.replace(".","").replace(",","."))
rdd1 = rdd.map(lambda line : splitAndMakeTuple(line) )

rdd2 = rdd1

car_rdd = rdd1.cartesian(rdd2)

def cal_dis(loc_1,loc_2):
    R=6373.0
    
    lat1=radians(loc_1[0])
    lon1=radians(loc_1[1])
    lat2=radians(loc_2[0])
    lon2=radians(loc_2[1])
    
    dlon = lon2-lon1
    dlat = lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2*atan2(sqrt(a),sqrt(1-a))
    distance=R*c
    return distance

dis_rdd = car_rdd.map(lambda x: ((x[0][0]+"-"+x[1][0]), cal_dis(x[0][1],x[1][1])))

dis_rdd = dis_rdd.filter(lambda x: x[1] > 0)

sortedRDD = dis_rdd.sortBy(lambda rdd :rdd[1],ascending=False)

rdd3 = sortedRDD.zipWithIndex().map(lambda x : (x[1],x[0]))

rdd4 = rdd3.map(lambda x: (x[0]+1,x[1]))

rdd4 = rdd4.filter(lambda x: x[0]%2 ==0)

rdd4 = rdd4.map(lambda x: x[1])
rdd4.take(5)

[('Taipei-Asunción', 19933.31283103459),
 ('Adamstown-Manama', 19876.418028651835),
 ('Madrid-Wellington', 19861.008143692714),
 ('Doha-Adamstown', 19856.630380263494),
 ('Mata-Utu-Niamey', 19833.509620968)]